In [1]:
import pandas as pd
from collections import Counter

student_courses = pd.read_excel("Courses_Enrollment.xlsx")  
course_info = pd.read_excel("UW_Courses_with_keywords.xlsx")         
course_info['Key Words'] = course_info['Key Words'].str.split(',')
course_keywords_map = course_info.set_index('Course ID')['Key Words'].to_dict()


In [ ]:
def generate_student_interests(student_courses, course_keywords_map):
    # Group by StudentID and aggregate courses for each student
    grouped_courses = student_courses.groupby('StudentID')['CourseID'].apply(list).reset_index()
    print(grouped_courses)
    
    student_interests = {}
    for index, row in grouped_courses.iterrows():
        student_id = row['StudentID']
        courses = row['CourseID']  # List of courses for this student
        keywords = []

        # Collect keywords for each course
        for course in courses:
            course = course.strip()
            if course in course_keywords_map:
                keywords.extend(course_keywords_map[course])

        # Count the frequency of each keyword
        keyword_counts = Counter(keywords)
        # Get the top 2 keywords
        top_keywords = [kw for kw, _ in keyword_counts.most_common(2)]

        # Store the top interests for each student
        student_interests[student_id] = top_keywords

    return student_interests

# Generate student interests
student_interests = generate_student_interests(student_courses, course_keywords_map)

# Create a DataFrame from the student interests dictionary
student_interests_df = pd.DataFrame.from_dict(student_interests, orient='index', columns=['Interest_1', 'Interest_2']).reset_index()
student_interests_df.rename(columns={'index': 'StudentID'}, inplace=True)

student_courses_aggregated = student_courses.groupby('StudentID')['CourseID'].apply(lambda x: ', '.join(x)).reset_index()
student_interests_df = pd.merge(student_interests_df, student_courses_aggregated, on='StudentID')
student_interests_df.rename(columns={'CourseID': 'Courses'}, inplace=True)

# Save to Excel
student_interests_df.to_excel('student_with_interest.xlsx', index=False)

# Display the DataFrame
print(student_interests_df)

    StudentID                                           CourseID
0       S0001  [CSE 556, CSE 550, CSE 533, E E 576, CSE 501, ...
1       S0002  [CSE 461, CSE 332, CSE 311, CSE 123, CSE 143, ...
2       S0003  [CSE 160, CSE 123, CSE 143, CSE 142, CSE 112, ...
3       S0004  [E E 530, E E 508, E E 505, E E 572, CSE 512, ...
4       S0005  [CSE 311, CSE 123, CSE 143, CSE 142, CSE 391, ...
..        ...                                                ...
195     S0196  [MATH 135, MATH 134, CSE 163, CSE 122, CSE 123...
196     S0197  [MATH 134, MATH 136, MATH 135, MATH 134, MATH ...
197     S0198  [E E 242, MATH 135, MATH 134, MATH 207, MATH 1...
198     S0199  [E E 462, E E 361, E E 215, MATH 136, MATH 135...
199     S0200  [MATH 209, MATH 207, MATH 125, MATH 124, MATH ...

[200 rows x 2 columns]
    StudentID                                   Interest_1  \
0       S0001                          Topology & Geometry   
1       S0002           Computation & Scientific Computing   
2       S0